# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df

,city,country,lat,lon,temp,humidity,cloudiness,wind speed
0,Barrow,US,71.29,-156.79,42.80,87,90,16.11
1,Hilo,US,19.73,-155.09,74.88,83,90,1.43
2,Guerrero Negro,MX,27.98,-114.06,79.48,50,0,14.43
3,Khatanga,RU,71.97,102.50,51.33,84,3,8.34
4,Shangqiu,CN,34.45,115.65,82.00,92,100,1.01
...,...,...,...,...,...,...,...,...
569,Esim,GH,4.87,-2.24,74.23,77,89,7.99
570,Requena,ES,39.49,-1.10,82.00,51,0,8.79
571,Henties Bay,NaN,-22.12,14.28,55.99,83,0,1.01
572,Gazojak,TM,41.19,61.40,83.73,21,68,9.91


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = city_data_df[["lat", "lon"]]
humidity = city_data_df["humidity"]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=3)
fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
perf_city_df = city_data_df.loc[(city_data_df["temp"] < 85)  
                              & (city_data_df["temp"] > 72)
                              & (city_data_df["wind speed"] < 5) 
                              & (city_data_df["cloudiness"] == 0)].dropna()
perf_city_df

,city,country,lat,lon,temp,humidity,cloudiness,wind speed
39,Lucapa,AO,-8.42,20.74,72.05,21,0,2.59
90,Umluj,SA,25.02,37.27,84.69,69,0,1.79
123,Dawlatabad,AF,36.41,64.91,77.63,25,0,2.28
150,Ovalle,CL,-30.60,-71.20,75.85,33,0,2.57
164,Yangi Marg`ilon,UZ,40.43,71.72,78.80,50,0,2.24
279,Bereda,ES,43.27,-7.54,83.53,41,0,3.00
426,Cogoleto,IT,44.39,8.64,77.63,78,0,2.24
505,Margate,GB,51.38,1.39,73.29,56,0,1.99
514,Köping,SE,59.51,15.99,73.11,60,0,2.24
531,Nisko,PL,50.52,22.14,75.09,69,0,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perf_city_df[["city", "country","lat", "lon"]]
hotel_df["hotel name"] = ""

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}


for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lon = row["lon"]
    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    response = response.json()
    
    try:
        hotel_df.loc[index, "hotel name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("missing")

hotel_df


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,city,country,lat,lon,hotel name
39,Lucapa,AO,-8.42,20.74,Residência Ulisses Dos Santos
90,Umluj,SA,25.02,37.27,Moon light Furnished Units
123,Dawlatabad,AF,36.41,64.91,WASIM ASGAR KHAN
150,Ovalle,CL,-30.60,-71.20,Hotel Altos de Tuqui
164,Yangi Marg`ilon,UZ,40.43,71.72,Ikathouse
279,Bereda,ES,43.27,-7.54,Pazo de Verdes
426,Cogoleto,IT,44.39,8.64,Grand Hotel Arenzano
505,Margate,GB,51.38,1.39,Premier Inn Margate hotel
514,Köping,SE,59.51,15.99,Best Western Hotel Scheele
531,Nisko,PL,50.52,22.14,Zajazd Pod Jesionami


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))